In [19]:
import tensorflow as tf
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

In [20]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
X = tf.placeholder(dtype=tf.float32, shape=(None, n + 1), name='X')
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

housing = fetch_california_housing()
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X_data = scaled_housing_data_plus_bias.astype(np.float32)
y_data = housing.target.reshape(-1, 1).astype(np.float32)

def fetch_batch(batch_index, batch_size):
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = X_data[indices] # not shown
    y_batch = y_data[indices] # not shown
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform((n+1, 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [21]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [22]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
#                 print("Epoch", epoch, "batch_index", batch_index, 
#                       "MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
    best_theta = theta.eval()
    final_mse = mse.eval(feed_dict={X: X_batch, y: y_batch})

In [23]:
file_writer.close()

In [24]:
ls -l tf_logs/run*

tf_logs/run-20180506134158:
total 816
-rw-r--r-- 1 root root 835089 May  6 13:42 events.out.tfevents.1525614170.04a0f2f0a265

tf_logs/run-20180506135132:
total 1728
-rw-r--r-- 1 root root 883390 May  6 13:51 events.out.tfevents.1525614693.04a0f2f0a265
-rw-r--r-- 1 root root 883647 May  6 13:51 events.out.tfevents.1525614699.04a0f2f0a265

tf_logs/run-20180506135209:
total 3176
-rw-r--r-- 1 root root 2738661 May  6 13:53 events.out.tfevents.1525614730.04a0f2f0a265


In [ ]:
!python -m tensorboard.main --logdir=tf_logs

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0506 15:34:46.364394 Reloader tf_logging.py:114] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
TensorBoard 1.5.1 at http://04a0f2f0a265:6006 (Press CTRL+C to quit)
